<a href="https://colab.research.google.com/github/arijitdas0211/ML_Python/blob/main/SpeechToText_and_TextToSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install pydub
!pip install ffmpeg-python
!pip install SpeechRecognition
!pip install gTTS

In [ ]:
from IPython.display import Javascript, Audio, display
from google.colab import output
from base64 import b64decode
from io import BytesIO
import ffmpeg
# import espeakng
import speech_recognition as sr
# import pyttsx3
from gtts import gTTS
import random
import string

In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

In [ ]:
def fix_riff_header(binary):
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]
  return riff

In [ ]:
def generate_random_filename(length: int = 5, extension: str = "wav") -> str:
    characters = string.ascii_letters + string.digits
    random_string = "".join(random.choice(characters) for _ in range(length))
    if extension:
        if "." in extension:
            pieces = extension.split(".")
            last_extension = pieces[-1]
            extension = last_extension
        return f"{random_string}.{extension}"
    return random_string

In [ ]:
def record(filename, sec):
  com = "Listening."
  display(Javascript(RECORD), com)
  s = output.eval_js('record(%d)' % (sec*1200))
  b = b64decode(s.split(',')[1])
  b = fix_riff_header(b)
  print(com.replace("Listening","Got it"))
  with open(filename,'wb') as f:
    f.write(b)
  return filename

In [ ]:
random_file = generate_random_filename()
file_name = record(f'inputAudio_{random_file}', 4)

In [ ]:
recognize = sr.Recognizer()
with sr.AudioFile(f'/content/{file_name}') as source:
    # listen for the data (load audio to memory)
    audio = recognize.record(source)

# while True:
try:
  audio_str = recognize.recognize_google(audio_data=audio)
  audio_str = audio_str.lower()
except Exception as err:
  audio_str = ''
  print('Recognizer error. Audio String is Empty.', err)

In [ ]:
if audio_str == '':
    audio_str = "Sorry, nothing to say. You haven't spoke anything."
else:
    print(f"Recognized text is:- {audio_str.capitalize()}\n")
    def gTTsAudio(filename=f'/content/outputAudio_{random_file}'):
      gtts_obj = gTTS(text=audio_str, lang='en', slow=True)
      gtts_obj.save(filename)
      return filename

Audio(gTTsAudio(), autoplay=True)